## Modeling

Create different models for each product's time-series


In [2]:
from data_exploitation.exports_analytics.data.scripts import time_series_generator
from data_exploitation.exports_analytics.exploratory_data_analysis.scripts import time_granularities_analysis, time_granularities_generator
from database_settings import postgres_utilities
import pandas as pd
import numpy as np
import math
import statsmodels.api as sm
from scipy.stats import pearsonr
from data_exploitation.exports_analytics.data_modeling.models import linear_regression, gam, prophet_1, prophet_2, prophet_3, arima, gradient_boosting
from tqdm.notebook import tqdm


Importing plotly failed. Interactive plots will not work.


### Get available headings

In [3]:
# Get available headings from peru_exports_sandbox
engine = postgres_utilities.engine()
available_headings = postgres_utilities.run_query_file(engine,
                                                       './../data/queries/read_available_headings.sql')
available_headings = pd.DataFrame(available_headings)
available_headings

Connected to Formatted Zone successfully!


,heading,description
0,0712909000,Otras hortalizas y mezclas de hortalizas corta...
1,0805291000,Tangelo (Citrus reticulata x Citrus paradisis)...
2,0810400000,"Arándanos rojos, mirtilos y demas frutos del g..."
3,0713359000,"Frijoles (fréjoles, porotos, alubias, judías) ..."
4,0805502100,"Limón (limón sutil, limón común, limón criollo..."
5,0810901000,"Granadilla, maracuya (parchita) y demas frutas..."
6,0702000000,Tomates frescos o refrigerados
7,0709999000,Otras hortalizas frescas o refrigeradas
8,0801220000,Nueces del Brasil sin cáscara
9,0710400000,"Maiz dulce incluso cocidos, congelados"


### Get models for each heading

#### Multi-step models


In [ ]:
# Set the observations options
observations_options = ['usd_kg', 'net_weight']
# Model's metrics dataframe
metrics_df = pd.DataFrame(columns=['heading', 'type', 'steps','model', 'rmse', 'mae'])

for heading in tqdm(available_headings['heading'].values):

    # Choose a heading
    chosen_heading = heading
    print('Analyzing {}...'.format(chosen_heading))
    # Iterate trough obsevation options
    for observations in observations_options:
        # Get the time-series
        ts = time_series_generator.generate(chosen_heading, observations)

        #-------------------------------------
        # Cyclic-time granularities generation
        #-------------------------------------
        time_granularities, priority = time_granularities_generator.generate(ts.start_date, ts.end_date)
        data = pd.concat([ts.data, time_granularities], axis=1)
        #-------------------------------------


        #-------------------------------------
        # Seasonality analysis
        #-------------------------------------
        # Define the granularities to analyze
        granularities = ['semester_in_year',
                         'quarter_in_year',
                         'month_in_year',
                         'month_in_semester',
                         'month_in_quarter',
                         'quarter_in_semester',
                         'day_in_month',
                         'day_in_week',
                         'week_in_year',
                         'week_in_semester',
                         'week_in_quarter',
                         'week_in_month']

        selected_granularities = []
        for g in granularities:
            # print('Analyzing {}...'.format(g))
            seasonality_analysis = time_granularities_analysis.analysis(cyclic_granularity=g,
                                            observations=observations,
                                            data=data)
            if seasonality_analysis.is_seasonal():
                selected_granularities.append(g)
        selected_granularities = [x for x in priority if x in selected_granularities]
        print("The time-series has {} seasonalities detected".format(len(selected_granularities)))
        #-------------------------------------

        #-------------------------------------
        # GMST decomposition
        #-------------------------------------

        # Get the time-series
        time_series = data[observations].values

        #--------------
        # SEASONALITIES
        #--------------
        # Initialize the seasonal patterns
        seasonal_patterns = {}

        # Get the pattern for every granularity
        for granularity in selected_granularities:

            # Get the label of every observation
            labels = data[granularity].values
            # Generate the pattern
            pattern = []
            for label in labels:
                value = np.mean(time_series[np.where(labels == label)])
                pattern.append(value)
            seasonal_patterns[granularity] = pattern
        #--------------

        #------
        # TREND
        #------
        # Initialize the parameters
        percentage = 0.00 # fraction of datapoints taken in the lowess smoothing
        statistic_th = 0.05 # threshold for the distance metric
        alpha = 0.05 # threshold for significance

        correlation_exists = True

        while(correlation_exists ## while they are correlated
              and
              not(math.isclose(percentage, 1)) # while the fraction for the lowess isn't 100%
        ):
            # Increase the percentage
            percentage = round(percentage + 0.01, 2)

            # Get the trend
            trend = sm.nonparametric.lowess(data[observations], data.index, frac=percentage)[:, 1]
            # Initialize the statistics summary
            pearson_correlations ={}
            # Make the comparisons
            for granularity, seasonal_pattern in seasonal_patterns.items():

                # Compute Pearson correlation coefficient and p-value
                analysis = pearsonr(seasonal_pattern, trend)

                pearson_correlations[granularity] = {'statistic':analysis.statistic, 'pvalue':analysis.pvalue}

            # Get the max correlation
            granularity, metrics = max(pearson_correlations.items(), key=lambda x: abs(x[1]['statistic']))
            statistic = metrics['statistic']

            # If the max correlation is smaller than the threshold, assess its significance
            if abs(statistic) < statistic_th:
                pvalue = pearson_correlations[granularity]['pvalue']/2
                # print('pvalue: {}'.format(pvalue))
                # If the pvalue is smaller than the threshold, it's significant
                if pvalue < alpha:
                    correlation_exists = False

        pearson_correlations = pd.DataFrame.from_dict(pearson_correlations, orient='index')
        print('Trend\'s smoothing fraction chosen: {}'.format(percentage))
        #------

        # Make an array with the decomposition results
        X = np.array([trend] + list(seasonal_patterns.values())).T

        #-------------------------------------

        #-------------------------------------
        # Multi-step models
        #-------------------------------------
        explanatory_variables = X
        target_variable = data[observations].values
        target_name = observations
        timepoints = data.index
        training_fraction = 0.8
        steps = [30, 60, 90]

        for s in steps:
            print('{} steps'.format(s))
            models = {'linear_regression':linear_regression,
                      'gam':gam,
                      'prophet_3':prophet_3,
                      'arima': arima,
                      'gradient_boosting':gradient_boosting}
            for model_name, model in models.items():
                m = model.model(explanatory_variables = explanatory_variables,
                                target_variable = target_variable,
                                target_name = target_name,
                                timepoints = timepoints,
                                training_fraction = training_fraction,
                                steps = s)
                assessment = m.test()
                metrics = {'heading': [chosen_heading],
                          'type':[observations],
                          'steps':[s],
                          'model': [model_name],
                          'rmse': [assessment['rmse']],
                          'mae':[assessment['mae']]}
                metrics_df = pd.concat([metrics_df, pd.DataFrame(metrics)], ignore_index = True)

# Save all
metrics_df.to_csv('./multi_step_models_metrics.csv')

#### Non-multistep models


In [5]:
# Set the observations options
observations_options = ['usd_kg', 'net_weight']
# Model's metrics dataframe
metrics_df = pd.DataFrame(columns=['heading', 'type','model', 'rmse', 'mae'])

for heading in tqdm(available_headings['heading'].values):

    # Choose a heading
    chosen_heading = heading
    print('Analyzing {}...'.format(chosen_heading))
    # Iterate trough obsevation options
    for observations in observations_options:
        # Get the time-series
        ts = time_series_generator.generate(chosen_heading, observations)

        #-------------------------------------
        # Cyclic-time granularities generation
        #-------------------------------------
        time_granularities, priority = time_granularities_generator.generate(ts.start_date, ts.end_date)
        data = pd.concat([ts.data, time_granularities], axis=1)
        #-------------------------------------


        #-------------------------------------
        # Seasonality analysis
        #-------------------------------------
        # Define the granularities to analyze
        granularities = ['semester_in_year',
                         'quarter_in_year',
                         'month_in_year',
                         'month_in_semester',
                         'month_in_quarter',
                         'quarter_in_semester',
                         'day_in_month',
                         'day_in_week',
                         'week_in_year',
                         'week_in_semester',
                         'week_in_quarter',
                         'week_in_month']

        selected_granularities = []
        for g in granularities:
            # print('Analyzing {}...'.format(g))
            seasonality_analysis = time_granularities_analysis.analysis(cyclic_granularity=g,
                                            observations=observations,
                                            data=data)
            if seasonality_analysis.is_seasonal():
                selected_granularities.append(g)
        selected_granularities = [x for x in priority if x in selected_granularities]
        print("The time-series has {} seasonalities detected".format(len(selected_granularities)))
        #-------------------------------------

        #-------------------------------------
        # GMST decomposition
        #-------------------------------------

        # Get the time-series
        time_series = data[observations].values

        #--------------
        # SEASONALITIES
        #--------------
        # Initialize the seasonal patterns
        seasonal_patterns = {}

        # Get the pattern for every granularity
        for granularity in selected_granularities:

            # Get the label of every observation
            labels = data[granularity].values
            # Generate the pattern
            pattern = []
            for label in labels:
                value = np.mean(time_series[np.where(labels == label)])
                pattern.append(value)
            seasonal_patterns[granularity] = pattern
        #--------------

        #------
        # TREND
        #------
        # Initialize the parameters
        percentage = 0.00 # fraction of datapoints taken in the lowess smoothing
        statistic_th = 0.05 # threshold for the distance metric
        alpha = 0.05 # threshold for significance

        correlation_exists = True

        while(correlation_exists ## while they are correlated
              and
              not(math.isclose(percentage, 1)) # while the fraction for the lowess isn't 100%
        ):
            # Increase the percentage
            percentage = round(percentage + 0.01, 2)

            # Get the trend
            trend = sm.nonparametric.lowess(data[observations], data.index, frac=percentage)[:, 1]
            # Initialize the statistics summary
            pearson_correlations ={}
            # Make the comparisons
            for granularity, seasonal_pattern in seasonal_patterns.items():

                # Compute Pearson correlation coefficient and p-value
                analysis = pearsonr(seasonal_pattern, trend)

                pearson_correlations[granularity] = {'statistic':analysis.statistic, 'pvalue':analysis.pvalue}

            # Get the max correlation
            granularity, metrics = max(pearson_correlations.items(), key=lambda x: abs(x[1]['statistic']))
            statistic = metrics['statistic']

            # If the max correlation is smaller than the threshold, assess its significance
            if abs(statistic) < statistic_th:
                pvalue = pearson_correlations[granularity]['pvalue']/2
                # print('pvalue: {}'.format(pvalue))
                # If the pvalue is smaller than the threshold, it's significant
                if pvalue < alpha:
                    correlation_exists = False

        pearson_correlations = pd.DataFrame.from_dict(pearson_correlations, orient='index')
        print('Trend\'s smoothing fraction chosen: {}'.format(percentage))
        #------

        # Make an array with the decomposition results
        X = np.array([trend] + list(seasonal_patterns.values())).T

        #-------------------------------------

        #-------------------------------------
        # Models
        #-------------------------------------
        explanatory_variables = X
        target_variable = data[observations].values
        target_name = observations
        timepoints = data.index
        training_fraction = 0.8

        models = {'prophet_1':prophet_1, 'prophet_2':prophet_2}
        for model_name, model in models.items():
            kwargs = {
                'target_variable': target_variable,
                'target_name': target_name,
                'timepoints': timepoints,
                'training_fraction': training_fraction
            }
            if model_name == 'prophet_2':
                kwargs['explanatory_variables'] = explanatory_variables
            m = model.model(**kwargs)
            assessment = m.test()
            metrics = {'heading': [chosen_heading],
                      'type':[observations],
                      'model': [model_name],
                      'rmse': [assessment['rmse']],
                      'mae':[assessment['mae']]}
            metrics_df = pd.concat([metrics_df, pd.DataFrame(metrics)], ignore_index = True)

# Save all
metrics_df.to_csv('./prophet_models_metrics.csv')

  0%|          | 0/47 [00:00<?, ?it/s]

Analyzing 0712909000...
The time-series has 8 seasonalities detected


10:10:09 - cmdstanpy - INFO - Chain [1] start processing


Trend's smoothing fraction chosen: 0.4


10:10:09 - cmdstanpy - INFO - Chain [1] done processing
10:10:10 - cmdstanpy - INFO - Chain [1] start processing
10:10:10 - cmdstanpy - INFO - Chain [1] done processing


...

11:26:30 - cmdstanpy - INFO - Chain [1] done processing
11:26:30 - cmdstanpy - INFO - Chain [1] start processing
11:26:30 - cmdstanpy - INFO - Chain [1] done processing


The time-series has 8 seasonalities detected


11:27:05 - cmdstanpy - INFO - Chain [1] start processing


Trend's smoothing fraction chosen: 0.25


11:27:05 - cmdstanpy - INFO - Chain [1] done processing
11:27:05 - cmdstanpy - INFO - Chain [1] start processing
11:27:05 - cmdstanpy - INFO - Chain [1] done processing
11:27:05 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
11:27:05 - cmdstanpy - INFO - Chain [1] start processing
11:27:08 - cmdstanpy - INFO - Chain [1] done processing
